<a href="https://colab.research.google.com/github/Harry-OBrien/Hex_bot/blob/master/rein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !python Hex.py "a=PNA;python /content/rl/COMP34111_e54158lm/agents/DefaultAgents/NaiveAgent.py" "a=PNA;python /content/rl/COMP34111_e54158lm/agents/DefaultAgents/NaiveAgent.py" -v

In [4]:
from zipfile import ZipFile
file_name = '/content/rl.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')


Done


In [5]:
%cd rl
%cd COMP34111_e54158lm

/content/rl
/content/rl/COMP34111_e54158lm


In [6]:
!ls

 agents  'Hex Game Engine Documentation.pdf'   Hex.py   README.md   src


In [7]:
%cd src

/content/rl/COMP34111_e54158lm/src


In [8]:
from Board import Board
from Game import Game

In [9]:
import numpy as np

In [10]:
class model():

  def __init__(self,board):
    self.board = board

  def board_rep(self,player_colour):  
    
    if player_colour =='R':
      player_num = 1.0
      opp_player_num =-1.0
    else:
      player_num = -1.0
      opp_player_num =1.0

    board_rep = board.print_board().split(',')
    int_rep =[]
    for i in range(board._board_size):
      str_row = board_rep[i]
      row = []
      for j in range(board._board_size):
        if str_row[j] == player_colour:
          row.append(player_num)
        else:
          row.append(0.0)
      row.insert(0,-1.0)
      row.insert(0,-1.0)
      row.append(-1.0)
      row.append(-1.0)
      int_rep.append(row)
 
    # pad_top_bottom
    vert_border = [1.0 for i in range(board._board_size)]
    vert_border.insert(0,0.0)
    vert_border.insert(0,0.0)
    vert_border.append(0.0)
    vert_border.append(0.0)

    int_rep.insert(0,vert_border)
    int_rep.insert(0,vert_border)
    int_rep.append(vert_border)
    int_rep.append(vert_border)

    board_array = np.array(int_rep)
    return board_array




In [11]:
import torch 
import numpy 
from torch import nn
import random

from collections import deque
from Tile import Tile
from Colour import Colour


In [12]:
def update_board(move,board,player):
 
  p1=board
  

  if player == 1:
    p1[move[0]+2,move[1]+2] = 1
   
  else:
    p1[move[0]+2,move[1]+2] = -1

  return p1


In [13]:
def _winner(board,player):
    
    if player == -1:
      board = board.transpose(0,1)

    else:
      i = 0
    # CHECK STARTING WITH THE TOP LINE IF THERE IS A CONNECTION TO BOTTOM
    visited = set()
    left = set()

    for j in range(len(board)):
        if(board[0][j] == player):
            left.add((0,j))

    while left:
        some = left.pop()
        visited.add(some)

        neighbours = _getNeighbours(board,some)
            
        for each in neighbours:
            if(board[each[0]][each[1]] == player):
                if (each[0] == len(board)-1):
                    return 1
                if each not in visited:
                    left.add(each)
    return 0




def _getNeighbours(board, ij):
    x = ij[1]
    y = ij[0]
    #clockwise neighbours of a x,y
    available = [True,True,True,True,True,True]
    I_DISPLACEMENTS = [-1, -1, 0, 1,  1, 0]
    J_DISPLACEMENTS = [0,  1,  1, 0, -1, -1]
    if(y == 0): 
        available[0] = False
        available[1] = False
    elif(y == len(board)-1):
        available[3] = False
        available[4] = False
    if(x == 0):
        available[4] = False
        available[5] = False
    elif(x == len(board)-1):
        available[1] = False
        available[2] = False
    result = []
    for i in range(6):
        if available[i]:
            result.append((y+I_DISPLACEMENTS[i], x+J_DISPLACEMENTS[i]))
    return result


In [ ]:
class Net(nn.Module):
  def __init__(self,board_size=15):
    super(Net, self).__init__()
    self.board_size = board_size
    self.conv = nn.Conv2d(1, 1 , 5, stride=1)
    self.conv2 = nn.Conv2d(1, 1, 1, stride=1)
    self.linear = nn.Linear(11**2,11**2)

  def forward(self,input):
   
    output = self.conv(input)
    output = self.conv2(output)
    

    output = torch.flatten(output,start_dim=2,end_dim=3)
    
    
    output = self.linear(output.squeeze(0).squeeze(0))
  
    return output


def board_rep():  
    player_num = 1
    opp_player_num = -1

    board_rep = board.print_board().split(',')
    int_rep =[]
    for i in range(board._board_size):
      str_row = board_rep[i]
      row = []
      for j in range(board._board_size):
        row.append(0.0)

      row.insert(0,-1.0)
      row.insert(0,-1.0)
      row.append(-1.0)
      row.append(-1.0)
      int_rep.append(row)
 
    # pad_top_bottom
    vert_border = [1.0 for i in range(board._board_size)]
    vert_border.insert(0,0.0)
    vert_border.insert(0,0.0)
    vert_border.append(0.0)
    vert_border.append(0.0)

    int_rep.insert(0,vert_border)
    int_rep.insert(0,vert_border)
    int_rep.append(vert_border)
    int_rep.append(vert_border)

    board_array = np.array(int_rep)
    return board_array



epochs = 100000

lo = torch.nn.MSELoss()
learning_rate = 1e-3

board = Board()
n = Net()
optimizer = torch.optim.Adam(n.parameters(), lr=learning_rate)
memory = []

for i in range(epochs):
  print(i)
  not_finished = 1
  state1 = board_rep()
  to_move = 1
  while(not_finished == 1):
   
    state1 = torch.from_numpy(state1)
    
    moves = n(state1.unsqueeze(0).unsqueeze(0).float())
    #Epsillon Greedy
    #RANDOM ACTION
    if (random.random() > 0.80):
      action = int(np.random.randint(0,121))
      move_to_make = (int(action/11),action%11)
      while(state1[move_to_make[0]+2,move_to_make[1]+2] != 0):
        action = int(np.random.randint(0,121))
        move_to_make = (int(action/11),action%11)
    #ELSE TAKE NEXT AVAILABLE BEST
    else:
      actions = torch.topk(moves, 121).indices
      action = int(actions[0])
      i = 0
      #MOVE IN BOARD TERMS
      move_to_make = (int(action/11),action%11)
     
      
      while((i < 121) and (state1[move_to_make[0]+2,move_to_make[1]+2] != 0)):
        action = int(actions[i])
        #MOVE IN BOARD TERMS
        move_to_make = (int(action/11),action%11)
        i+=1

    new_state = update_board(move_to_make,state1,to_move)
    #updated board
    new_state = new_state.float()
    #is the game done?
    done = 0
    #if can win by playing that move, reward is positive
    if _winner(new_state,to_move):
      print(new_state)
      reward = 1
      done = 1
      not_finished = 0
    else:
      reward = 0
    
    #only log the states of moves we've made
    if to_move==1:
      memory.append((state1,new_state,action,reward,done))
      # if done == 1:
      #   print('WEEEEE WONNNNNNN')
      #   for i in range(10):
      #     print()
  
    state1 = new_state.numpy()
    #UPDATE BATCH 
    if len(memory) > 200 and to_move==1:
      #random sample
      minibatch = random.sample(memory, 10)
      
      our_states = torch.cat([state1.unsqueeze(0).unsqueeze(0).float() for (state1,new_state,action,reward,done) in minibatch])
      opp_states = torch.cat([new_state.unsqueeze(0).unsqueeze(0).float() for (state1,new_state,action,reward,done) in minibatch])
      done  = torch.Tensor([done for (state1,new_state,action,reward,done) in minibatch])
      reward = torch.Tensor([reward for (state1,new_state,action,reward,done) in minibatch])
      action = torch.Tensor([action for (state1,new_state,action,reward,done) in minibatch])
      
      #OUR MOVE VALUES
      q = n(our_states)
      #OPPONENT MOVES FROM S_T+1
      #DONT UPDATE OR CALCULATE GRADIENTS
      with torch.no_grad():
        opp_q = n(opp_states)
      opps_best = torch.max(opp_q,dim=2).values
      target = reward.unsqueeze(1) - opps_best
      X = q.squeeze(1).gather(dim=1,index=action.long().unsqueeze(1))

      loss = lo(X,target)
      # print(loss)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    #CHANGE PLAYER  
    to_move *= -1


Streaming output truncated to the last 5000 lines.
        [-1., -1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,
         -1.],
        [-1., -1.,  0., -1., -1., -1., -1.,  0.,  1.,  0.,  0.,  0.,  1., -1.,
         -1.],
        [ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
          0.],
        [ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
          0.]])
172391
tensor([[ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
          0.],
        [ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
          0.],
        [-1., -1.,  0.,  0.,  1.,  0.,  1.,  1., -1.,  1.,  1., -1.,  0., -1.,
         -1.],
        [-1., -1.,  0.,  0.,  1.,  0.,  1., -1.,  1.,  0., -1.,  0.,  0., -1.,
         -1.],
        [-1., -1., -1., -1.,  0.,  1., -1., -1.,  1.,  1., -1.,  0., -1., -1.,
         -1.],
        [-1., -1., -1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1., -1.,  1., -1.,
         -1.],
 

In [15]:
(int(11/11),11%11)

(1, 0)

In [16]:
random.sample([[1,2,3,4,5],[3,4,5,6,6],[4,5,5,5,5]],2)

[[3, 4, 5, 6, 6], [1, 2, 3, 4, 5]]